In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from category_encoders import MEstimateEncoder
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
filepath = "/Users/balqeesjabri/Downloads/titanic.csv"
titanic_data = pd.read_csv(filepath)

In [2]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Assuming 'preds' contains the predicted labels for the validation set, and 'y_valid' contains the true labels
def evaluate_classification_model(y_valid, preds):
    # Calculate precision, recall, and F1-score
    precision = precision_score(y_valid, preds)
    recall = recall_score(y_valid, preds)
    f1 = f1_score(y_valid, preds)
    
    # Calculate the confusion matrix
    conf_matrix = confusion_matrix(y_valid, preds)
    
    # Print the results
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)
    print("Confusion Matrix:")
    print(conf_matrix)




In [3]:
titanic_data.select_dtypes(["object"]).nunique()

Name        891
Sex           2
Ticket      681
Cabin       147
Embarked      3
dtype: int64

In [4]:
# we will drop the name column since this feature does not add any meaningful information and is unique for each sample,
# it is unlikely to help your model make accurate predictions.
titanic_data = titanic_data.drop(columns=['Name', 'PassengerId'])

In [5]:
y = titanic_data.Survived
X = titanic_data.drop(['Survived'], axis=1)

In [6]:
categorical_cols = [colname for colname in titanic_data.columns if titanic_data[colname].dtype == "object"]
print(categorical_cols)

['Sex', 'Ticket', 'Cabin', 'Embarked']


In [7]:
titanic_data.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

In [8]:
titanic_data.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

In [9]:
from sklearn.model_selection import train_test_split

X = titanic_data.drop(columns="Survived")
y = titanic_data["Survived"]

# Split the data into training and encoding sets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, random_state=0)


In [10]:
# from category_encoders import MEstimateEncoder

# # Choose a set of features to encode and a value for m
# encoder = MEstimateEncoder(cols=categorical_cols, m=5.0)

# # # Fit the encoder on the encoding set
# # encoder.fit(X_valid_full, y_valid)

# # # Encode the encoding set
# # X_train = encoder.transform(X_train_full, y_train)


In [11]:
# filter categorical cols only
categorical_features = [colname for colname in X_train_full.columns if X_train_full[colname].dtype == "object"]

# Select numerical columns
numerical_features = [colname for colname in X_train_full.columns if X_train_full[colname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_features + numerical_features
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [12]:
# # # import library
# from collections import Counter
# from imblearn.over_sampling import RandomOverSampler

# ros = RandomOverSampler(random_state=42)
# # fit predictor and target variablex_ros, y_ros = ros.fit_resample(x, y)
# X_ros, y_ros = ros.fit_resample(X_train, y_train)

# print('Original dataset shape', Counter(y_train))
# print('Resample dataset shape', Counter(y_ros))

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from imblearn.under_sampling import TomekLinks
from xgboost import XGBClassifier

# Assuming you have defined your numerical_cols and categorical_cols appropriately

# Step 1: Create transformers for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', MEstimateEncoder(m=5.0))  # Use MEstimateEncoder here
])

# Step 2: Create a ColumnTransformer to apply different transformers to different feature types
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
])



# define model 
my_model = XGBClassifier(n_estimators=850, learning_rate=0.06)

# Step 5: Create the pipeline 
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', my_model)
])
# This line fits the pipeline to the training data,
my_pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   MEstimateEncoder(m=5.0))]),
                                                  ['Sex', 'Ticket', 'Cabin',
                                                   'Embarked'])])),
                ('model',
                 XG...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=0.06,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=850,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=None, ...))])

In [14]:
# Preprocess the data using the preprocessor
X_train_preprocessed = preprocessor.transform(X_train)
X_valid_preprocessed = preprocessor.transform(X_valid)

In [15]:
X_df = pd.DataFrame(X_train_preprocessed)
# Split the transformed data 
X_train_df, X_valid_df, y_train, y_valid = train_test_split(X_df, y_train, random_state=50)

In [16]:
# define model
my_model = XGBClassifier(n_estimators=850, learning_rate=0.07)
# fit model
my_model.fit(X_train_df, y_train, 
             early_stopping_rounds=7, 
             eval_set=[(X_valid_df, y_valid)], 
             verbose=False)

/usr/local/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.07, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=850, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [17]:
# Step 5: Create the pipeline 
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', my_model)
])

In [18]:
from sklearn.model_selection import cross_val_predict
preds_cv = cross_val_predict(my_pipeline, X, y, cv=5)  # Replace X and y with your features and target data

evaluate_classification_model(y, preds_cv)

Precision: 0.685459940652819
Recall: 0.6754385964912281
F1-score: 0.6804123711340206
Confusion Matrix:
[[443 106]
 [111 231]]


target enccoding only:
Precision: 0.685459940652819
Recall: 0.6754385964912281
F1-score: 0.6804123711340206
Confusion Matrix:
[[443 106]
 [111 231]]
